In [122]:
#!pip install statsmodels



import pandas as pd
import numpy as np
from math import sqrt
import holidays
import pmdarima as pm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from datetime import datetime
from holidays import Germany
import geopy
from pytz import UTC
from geopy.geocoders import Nominatim


from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#from config.paths import SAMPLE_DIR, ROOT_DIR

# Set your seed
seed = np.random.seed(42)

In [123]:
stations_info = pd.read_csv('data/stations.csv')
stations_data = pd.read_csv('data/sample/stations/2023-05-03-stations.csv')

In [124]:
# Load the dataset
price_mdata = pd.read_csv('/Users/morad/Downloads/diesel.csv')

# Pick a random station
random_station = '72f7a2d4-1b77-4556-a45e-8abfb804fb3b'

# Create a new DataFrame that contains only the rows of the randomly chosen station
price_mdata = price_mdata[price_mdata['station'] == random_station]
price_mdata = price_mdata[['date', 'diesel']]
price_mdata.reset_index(drop=True, inplace=True)


In [125]:
# Limit data to one month duration
start_date = pd.to_datetime('2020-01-30', utc=True)
end_date = pd.to_datetime('2023-05-31', utc=True)

start_date_f = pd.to_datetime('2023-06-01', utc=True)
end_date_f = pd.to_datetime('2023-06-14', utc=True)

# Ensure 'date' column is in pandas datetime format
price_mdata['date'] = pd.to_datetime(price_mdata['date'])

mask = (price_mdata['date'] >= start_date) & (price_mdata['date'] <= end_date)
price_mdata = price_mdata.loc[mask]



In [126]:
crude_oil = pd.read_csv('/Users/morad/Desktop/fill_crude_oil_2014_2023.csv')

In [127]:

holiday_data = pd.read_csv('data/sample/school_holiday_data_2014_2024.csv')
holiday_df = holiday_data[['date', 'NW']].copy()  # Create a copy of the DataFrame

# Convert boolean values to integers (0s and 1s)
holiday_df['NW'] = holiday_df['NW'].astype(int)

# Assign 'NW' column to 'holiday' column
holiday_df.rename(columns={'date': 'date', 'NW': 'holiday'}, inplace=True)


In [128]:


# Convert the 'date' column in holiday_df to datetime
holiday_df['date'] = pd.to_datetime(holiday_df['date'])

# Localize the 'date' column in holiday_df to a specific timezone
holiday_df['date'] = holiday_df['date'].dt.tz_localize(UTC)




In [129]:
# Convert 'date' in crude_oil to datetime, and set the timezone to UTC
crude_oil['date'] = pd.to_datetime(crude_oil['date']).dt.tz_localize('UTC')

# Set 'date' as index
crude_oil.set_index('date', inplace=True)

# Resample 'crude_oil' to match the frequency of 'price_mdata' (assuming it's hourly)
crude_oil = crude_oil.resample('H').ffill()

# Reset the index
crude_oil.reset_index(inplace=True)

# Now 'date' in both dataframes are in datetime format and have the same timezone
# So, you can merge them on 'date'
merged_exog = pd.merge( crude_oil,holiday_df, on='date', how='left')



In [130]:
# Fill missing crude_oil_price values
merged_exog = merged_exog.fillna(method='ffill')
merged_exog = merged_exog.fillna(method='bfill')

In [131]:
# Convert both date columns to datetime

price_mdata['date'] = pd.to_datetime(price_mdata['date'], utc=True)


In [132]:
# # Merge the DataFrames on the 'date' column
merged_df = pd.merge(price_mdata, merged_exog, on='date', how = 'left')

In [133]:
merged_df.head()

,date,diesel,crude_oil_price,holiday
0,2020-01-30 00:00:00+00:00,1.2690,57.72,0.0
1,2020-01-30 01:00:00+00:00,1.2690,57.72,0.0
2,2020-01-30 02:00:00+00:00,1.2690,57.72,0.0
3,2020-01-30 03:00:00+00:00,1.2690,57.72,0.0
4,2020-01-30 04:00:00+00:00,1.3065,57.72,0.0


In [134]:
# import itertools

# ###SARIMAX###

# #Constructs all possible parameter combinations.
# p = d = q = range(0,2)
# pdq = list(itertools.product(p,d,q))

# seasonal_pdq = [(x[0],x[1],x[2],24) for x in list(itertools.product(p,d,q))]


In [135]:
# def sarimax_function(data,pdq,s_pdq):

#     """
#     The function uses a brute force approach to apply all possible pdq combinations and evaluate the model
#     """

#     result_list = []
#     for param in pdq:
#         for s_param in s_pdq:

#             model = sm.tsa.statespace.SARIMAX(data, order=param, seasonal_order=s_param,
#             enforce_invertibility=False,enforce_stationarity=False)

#             results = model.fit()
#             result_list.append([param,s_param,results.aic])
#             print("ARIMA Parameters: {} x: {}. AIC: {}".format(param,s_param,results.aic))

#     return result_list,results

In [136]:
# result_list,results = sarimax_function(merged_df["diesel"],pdq,seasonal_pdq)

In [137]:
# results_dataframe = pd.DataFrame(result_list, columns=["dpq","s_dpq","aic"]).sort_values(by="aic")
# results_dataframe.head()

In [138]:

model = sm.tsa.statespace.SARIMAX(
    endog=merged_df["diesel"],
    #exog=merged_df[['holiday', 'crude_oil_price']],
    order=(1, 1, 1),
    seasonal_order=(2, 1, 2, 12), # it should be 1, 1, 0, 24 but the kernel was crashed 
    enforce_invertibility=True,
    enforce_stationarity=True
).fit()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.20277D+00    |proj g|=  2.00026D+01

At iterate    5    f= -2.60407D+00    |proj g|=  5.48811D+00

At iterate   10    f= -2.65711D+00    |proj g|=  4.24938D-01

At iterate   15    f= -2.65733D+00    |proj g|=  3.33944D-01

At iterate   20    f= -2.66361D+00    |proj g|=  2.31745D-01

At iterate   25    f= -2.66728D+00    |proj g|=  1.66505D-01

At iterate   30    f= -2.66957D+00    |proj g|=  3.86783D-02

At iterate   35    f= -2.67127D+00    |proj g|=  2.08554D-02

At iterate   40    f= -2.67130D+00    |proj g|=  1.14910D-02

At iterate   45    f= -2.67258D+00    |proj g|=  1.02605D-01


/Users/morad/jurassic-juice-juggler/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals




At iterate   50    f= -2.68105D+00    |proj g|=  4.50348D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     63      1     0     0   4.503D-02  -2.681D+00
  F =  -2.6810454760551270     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


In [139]:
prediction = merged_df[['date', 'diesel']]

In [140]:
prediction.head()

,date,diesel
0,2020-01-30 00:00:00+00:00,1.2690
1,2020-01-30 01:00:00+00:00,1.2690
2,2020-01-30 02:00:00+00:00,1.2690
3,2020-01-30 03:00:00+00:00,1.2690
4,2020-01-30 04:00:00+00:00,1.3065


In [141]:
import pytz

prediction['date'] = pd.to_datetime(prediction['date']).dt.tz_convert(pytz.UTC)
original_index = prediction.set_index('date').index
last_date = original_index[-1]
start_date = last_date + pd.DateOffset(hours=1)
end_date = pd.to_datetime('2023-06-30').tz_localize('UTC')
forecast_index = pd.date_range(start=start_date, end=end_date, freq='H', tz='UTC')

data_reindexed = prediction.reindex(original_index.union(forecast_index)).copy()
start_index = data_reindexed.index.get_indexer([start_date])[0]
forecast_values = model.predict(start=start_index, end=start_index + len(forecast_index) - 1, dynamic=False)

# Step 1: Get the datetime index corresponding to the forecast_values from data_reindexed
forecast_index = data_reindexed.index[start_index : start_index + len(forecast_values)]

# Step 2: Assign this new datetime index to forecast_values
forecast_values.index = forecast_index

# Step 3: Now try the assignment again
data_reindexed.loc[forecast_index, 'forecast'] = forecast_values

# Convert forecast_values to DataFrame and rename the column
forecast_df = forecast_values.to_frame().rename(columns={'predicted_mean': 'diesel'})

# Set 'date' as index for merged_df
merged_df.set_index('date', inplace=True)

# Now perform the concatenation
result_df = pd.concat([merged_df, forecast_df])




/var/folders/8x/cz487gy55dx6vlxs84h83w7h0000gn/T/ipykernel_14498/1100212317.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [142]:
result_df.to_csv('data/forecast_sarimax.csv',index=True)


In [143]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29857 entries, 2020-01-30 00:00:00+00:00 to 2023-06-30 00:00:00+00:00
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   diesel           29857 non-null  float64
 1   crude_oil_price  29137 non-null  float64
 2   holiday          29137 non-null  float64
dtypes: float64(3)
memory usage: 933.0 KB


In [144]:
import plotly.graph_objects as go

fig = go.Figure()

# Add a trace for the diesel column
fig.add_trace(go.Scatter(
    x=result_df.index,
    y=result_df['diesel'],
    mode='lines',
    name='diesel',
))

# Define your date range
start_date = '2023-06-01'
end_date = '2023-06-10'

fig.update_layout(
    title='Diesel Prices over Time',
    xaxis_title='Date',
    xaxis=dict(range=[start_date, end_date]),  # Set x-axis limits
    yaxis_title='Diesel Price',
    yaxis=dict(range=[1.45, 1.7]),  # Set y-axis limits
    autosize=False,
    width=800,
    height=500,
)

fig.show()
